<a href="https://colab.research.google.com/github/aayurchik/app_OpenWeatherMap/blob/main/analysis_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from multiprocessing import Pool, cpu_count
import requests
import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()  # чтобы asyncio работал в Jupyter/Colab

# *Загрузка csv*

In [ ]:
# Загрузка csv
# Реальные средние температуры (примерные данные) для городов по сезонам
seasonal_temperatures = {
    "New York": {"winter": 0, "spring": 10, "summer": 25, "autumn": 15},
    "London": {"winter": 5, "spring": 11, "summer": 18, "autumn": 12},
    "Paris": {"winter": 4, "spring": 12, "summer": 20, "autumn": 13},
    "Tokyo": {"winter": 6, "spring": 15, "summer": 27, "autumn": 18},
    "Moscow": {"winter": -10, "spring": 5, "summer": 18, "autumn": 8},
    "Sydney": {"winter": 12, "spring": 18, "summer": 25, "autumn": 20},
    "Berlin": {"winter": 0, "spring": 10, "summer": 20, "autumn": 11},
    "Beijing": {"winter": -2, "spring": 13, "summer": 27, "autumn": 16},
    "Rio de Janeiro": {"winter": 20, "spring": 25, "summer": 30, "autumn": 25},
    "Dubai": {"winter": 20, "spring": 30, "summer": 40, "autumn": 30},
    "Los Angeles": {"winter": 15, "spring": 18, "summer": 25, "autumn": 20},
    "Singapore": {"winter": 27, "spring": 28, "summer": 28, "autumn": 27},
    "Mumbai": {"winter": 25, "spring": 30, "summer": 35, "autumn": 30},
    "Cairo": {"winter": 15, "spring": 25, "summer": 35, "autumn": 25},
    "Mexico City": {"winter": 12, "spring": 18, "summer": 20, "autumn": 15},}

# Сопоставление месяцев с сезонами
month_to_season = {12: "winter", 1: "winter", 2: "winter",
                   3: "spring", 4: "spring", 5: "spring",
                   6: "summer", 7: "summer", 8: "summer",
                   9: "autumn", 10: "autumn", 11: "autumn"}

# Генерация данных о температуре
def generate_realistic_temperature_data(cities, num_years=10):
    dates = pd.date_range(start="2010-01-01", periods=365 * num_years, freq="D")
    data = []

    for city in cities:
        for date in dates:
            season = month_to_season[date.month]
            mean_temp = seasonal_temperatures[city][season]
            # Добавляем случайное отклонение
            temperature = np.random.normal(loc=mean_temp, scale=5)
            data.append({"city": city, "timestamp": date, "temperature": temperature})

    df = pd.DataFrame(data)
    df['season'] = df['timestamp'].dt.month.map(lambda x: month_to_season[x])
    return df

# Генерация данных
data = generate_realistic_temperature_data(list(seasonal_temperatures.keys()))
data.to_csv('temperature_data.csv', index=False)


# загрузка CSV как входных данных (как будет в Streamlit)
df = pd.read_csv("temperature_data.csv")

# Приводим timestamp к datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])

# 1. **Анализ исторических данных**:
   - Вычислить **скользящее среднее** температуры с окном в 30 дней для сглаживания краткосрочных колебаний.
   - Рассчитать среднюю температуру и стандартное отклонение для каждого сезона в каждом городе.
   - Выявить аномалии, где температура выходит за пределы $ \text{среднее} \pm 2\sigma $.
   - Попробуйте распараллелить проведение этого анализа. Сравните скорость выполнения анализа с распараллеливанием и без него.

In [ ]:
# Функция анализа одного города
def analyze_city(city_df):
    city_df = city_df.sort_values("timestamp")
    # Скользящее среднее и стандартное отклонение за 30 дней
    city_df["rolling_mean_30d"] = city_df["temperature"].rolling(30, min_periods=1).mean()
    city_df["rolling_std_30d"] = city_df["temperature"].rolling(30, min_periods=1).std()
    # Рассчитать среднюю температуру и стандартное отклонение для каждого сезона в каждом городе.
    stats = city_df.groupby("season")["temperature"].agg(["mean", "std"]).reset_index()
    city_df = city_df.merge(stats, on="season", how="left")
    # Выявить аномалии, где температура выходит за пределы  среднее±2σ
    city_df["is_anomaly"] = (city_df["temperature"] < city_df["mean"] - 2*city_df["std"]) | (city_df["temperature"] > city_df["mean"] + 2*city_df["std"])
    return city_df

# Последовательный вариант
start_seq = time.time()
sequential_result = pd.concat([analyze_city(group) for _, group in df.groupby("city")])
seq_time = time.time() - start_seq

# Параллельный вариант
start_par = time.time()
with Pool(cpu_count()) as pool:
    parallel_result = pd.concat(pool.map(analyze_city, [g for _, g in df.groupby("city")]))
par_time = time.time() - start_par

# Вывод
print("Скользящее среднее и std по 30 дням (первые 5 строк):")
print(parallel_result[["city","timestamp","temperature","rolling_mean_30d","rolling_std_30d"]].head(), end="\n\n")
seasonal_stats = parallel_result.groupby(["city","season"]).agg(season_mean_temp=("mean","first"),season_std_temp=("std","first")).reset_index()
print("Сезонная статистика по городам (первые 5 строк):")
print(seasonal_stats.head(), end="\n\n")
print("Аномалии (первые 5 строк):")
print(parallel_result[["city","timestamp","temperature","is_anomaly"]].head(), end="\n\n")
print(f"Время выполнения последовательного анализа: {seq_time:.2f} сек")
print(f"Время выполнения параллельного анализа: {par_time:.2f} сек")


Скользящее среднее и std по 30 дням (первые 5 строк):
      city  timestamp  temperature  rolling_mean_30d  rolling_std_30d
0  Beijing 2010-01-01    -7.003093         -7.003093              NaN
1  Beijing 2010-01-02     5.427891         -0.787601         8.790033
2  Beijing 2010-01-03    -4.783678         -2.119627         6.629873
3  Beijing 2010-01-04    -8.363924         -3.680701         6.249103
4  Beijing 2010-01-05     2.695251         -2.405511         6.117109

Сезонная статистика по городам (первые 5 строк):
      city  season  season_mean_temp  season_std_temp
0  Beijing  autumn         16.099292         5.203604
1  Beijing  spring         13.082266         5.092471
2  Beijing  summer         27.145010         5.168124
3  Beijing  winter         -1.940413         4.910997
4   Berlin  autumn         11.116922         5.197808

Аномалии (первые 5 строк):
      city  timestamp  temperature  is_anomaly
0  Beijing 2010-01-01    -7.003093       False
1  Beijing 2010-01-02     5.42

In [ ]:
# Сначала прогоняем analyze_city для всех городов
full_result = pd.concat([analyze_city(group) for _, group in df.groupby("city")])
# Смотрим, есть ли хотя бы одна аномалия
num_anomalies = full_result["is_anomaly"].sum()
print(f"Всего аномалий в датасете: {num_anomalies}")
# вывести первые пару аномалий
anomalies = full_result[full_result["is_anomaly"]]
print("Первые 5 аномалий:")
print(anomalies.head())

Всего аномалий в датасете: 2446
Первые 5 аномалий:
        city  timestamp  temperature  season  rolling_mean_30d  \
45   Beijing 2010-02-15    10.296674  winter         -0.302975   
55   Beijing 2010-02-25     8.492467  winter          0.316137   
103  Beijing 2010-04-14     1.580806  spring         11.316172   
111  Beijing 2010-04-22     1.180995  spring         11.903849   
160  Beijing 2010-06-10    42.250482  summer         16.939747   

     rolling_std_30d       mean       std  is_anomaly  
45          5.033413  -1.940413  4.910997        True  
55          5.164791  -1.940413  4.910997        True  
103         4.493819  13.082266  5.092471        True  
111         5.585082  13.082266  5.092471        True  
160         9.533882  27.145010  5.168124        True  


# 2. **Мониторинг текущей температуры**:
   - Подключить OpenWeatherMap API для получения текущей температуры города. Для получения API Key (бесплатно) надо зарегистрироваться на сайте. Обратите внимание, что API Key может активироваться только через 2-3 часа, это нормально. Посему получите ключ заранее.
   - Получить текущую температуру для выбранного города через OpenWeatherMap API.
   - Определить, является ли текущая температура нормальной, исходя из исторических данных для текущего сезона.
   - Данные на самом деле не совсем реальные (сюрпрайз). Поэтому на момент эксперимента погода в Берлине, Каире и Дубае была в рамках нормы, а в Пекине и Москве аномальная. Протестируйте свое решение для разных городов.
   - Попробуйте для получения текущей температуры использовать синхронные и асинхронные методы. Что здесь лучше использовать?

In [ ]:
#  Функция синхронного запроса текущей температуры для одного города
def get_current_temp_sync(city_name, api_key):
    # формируем URL запроса к OpenWeatherMap API
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}&units=metric"
    response = requests.get(url)
    # обработка ошибок
    if response.status_code != 200:
        return {"error": response.json().get("message", "Unknown error")}
    data = response.json()
    # возвращаем текущую температуру и описание погоды
    return {"temp": data["main"]["temp"], "description": data["weather"][0]["description"]}

# Асинхронная функция запроса текущей температуры для одного города
async def fetch_temp_async(session, city, api_key):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    async with session.get(url) as resp:
        # проверка ошибок ответа
        if resp.status != 200:
            return city, {"error": await resp.json()}
        data = await resp.json()
        # возвращаем текущую температуру и описание погоды
        return city, {"temp": data["main"]["temp"], "description": data["weather"][0]["description"]}

# Асинхронная функция для получения температуры сразу для нескольких городов
async def get_multiple_temps_async(cities, api_key):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_temp_async(session, city, api_key) for city in cities]
        results = await asyncio.gather(*tasks)
        return dict(results)

# Функция проверки аномалии на основе исторических данных
def check_anomaly_from_history(city, temp, historical_df):
    # определяем текущий месяц и сезон
    now = datetime.now()
    season = month_to_season[now.month]
    # извлекаем исторические статистики для данного города и сезона
    stats = historical_df[(historical_df["city"] == city) & (historical_df["season"] == season)].iloc[0]
    mean = stats["mean"]
    std = stats["std"]
    # температура считается аномальной, если выходит за пределы mean ± 2*std
    is_anomaly = (temp < mean - 2*std) or (temp > mean + 2*std)
    return is_anomaly, season

# Настройка API и список городов
api_key = "932cf927ad1e580b34ca7784b02376ff"
cities = ["Moscow", "Berlin", "Cairo", "Dubai", "Beijing"]

# Синхронный запрос всех городов и измерение времени
start_sync = time.time()  # начало таймера
sync_results = {}
for city in cities:
    sync_results[city] = get_current_temp_sync(city, api_key)
sync_time = time.time() - start_sync  # время выполнения синхронного метода

# Вывод результатов синхронного метода с проверкой аномалий
print("Синхронный метод:")
for c, info in sync_results.items():
    if "error" in info:
        print(c, "Ошибка:", info["error"])  # обработка ошибок запроса
    else:
        temp = info["temp"]
        desc = info["description"]
        # проверка аномальности через исторические данные
        is_anomaly, season = check_anomaly_from_history(c, temp, parallel_result)
        status = "аномальная" if is_anomaly else "в пределах нормы"
        # вывод текущей температуры, сезона и статуса
        print(f"{c}: {temp}°C, {desc}, сезон {season}, {status}")
print(f"Время выполнения синхронного метода: {sync_time:.2f} сек\n")  # сравнение методов

# Асинхронный запрос всех городов и измерение времени
start_async = time.time()  # начало таймера
async_results = asyncio.run(get_multiple_temps_async(cities, api_key))
async_time = time.time() - start_async  # время выполнения асинхронного метода

# Вывод результатов асинхронного метода с проверкой аномалий
print("Асинхронный метод:")
for c, info in async_results.items():
    if "error" in info:
        print(c, "Ошибка:", info["error"])  # обработка ошибок запроса
    else:
        temp = info["temp"]
        desc = info["description"]
        # проверка аномальности через исторические данные
        is_anomaly, season = check_anomaly_from_history(c, temp, parallel_result)
        status = "аномальная" if is_anomaly else "в пределах нормы"
        # вывод текущей температуры, сезона и статуса
        print(f"{c}: {temp}°C, {desc}, сезон {season}, {status}")
print(f"Время выполнения асинхронного метода: {async_time:.2f} сек\n")  # сравнение методов

# Вывод, какой метод лучше по скорости для этого числа городов
if sync_time < async_time:
    print("синхронный метод быстрее")
else:
    print("асинхронный метод быстрее")


Синхронный метод:
Moscow: -4.76°C, overcast clouds, сезон winter, в пределах нормы
Berlin: -3.55°C, clear sky, сезон winter, в пределах нормы
Cairo: 20.42°C, scattered clouds, сезон winter, в пределах нормы
Dubai: 22.96°C, clear sky, сезон winter, в пределах нормы
Beijing: -9.06°C, clear sky, сезон winter, в пределах нормы
Время выполнения синхронного метода: 0.70 сек

Асинхронный метод:
Moscow: -4.76°C, overcast clouds, сезон winter, в пределах нормы
Berlin: -3.55°C, clear sky, сезон winter, в пределах нормы
Cairo: 20.42°C, scattered clouds, сезон winter, в пределах нормы
Dubai: 22.96°C, clear sky, сезон winter, в пределах нормы
Beijing: -9.06°C, clear sky, сезон winter, в пределах нормы
Время выполнения асинхронного метода: 0.24 сек

асинхронный метод быстрее
